In [1]:
import psycopg2
import shapely

#TESTER DE FAIRE AVEC ÇA : http://postgis.net/docs/ST_ApproximateMedialAxis.html

con = psycopg2.connect(host="localhost", port=54320, database="vtile", user='ubuntu')
cur = con.cursor()

print('on supprime la table de restriction des données sur la TAN')
cmd = "drop table tan;"
cur.execute(cmd)

cmd = """
select * 
into tan
from lines
where code in ('2', '3')
and operator = 'SEMITAN'
"""
cur.execute(cmd)

print('on supprime la table de travail')
cmd = "drop table tan_tmp;"
cur.execute(cmd)

print('on crée la table de travail en éclatant les segments')
cmd = """
select id || '-' || (ST_Dump(geom)).path[1] as id , 
code, name, colour, mode, network,
(ST_Dump(geom)).path[1], 
(ST_Dump(geom)).geom, ST_Buffer((ST_Dump(geom)).geom, 20), 
st_length((ST_Dump(geom)).geom) 
into tan_tmp
from tan;"""
cur.execute(cmd)

print("on supprime les segments de chaque ligne qui est inclus dans un buffer de 20m d'un autre segment plus long")
cmd ="""
delete from tan_tmp
where id in (
select distinct id from 
(
	(select code, geom, st_buffer, st_length from tan_tmp) tmp1
	left join 
	(select * from tan_tmp) tmp2
	on tmp1.code = tmp2.code
	and tmp1.st_length > tmp2.st_length
	and st_contains(tmp1.st_buffer, tmp2.geom)

) 
tmp);
"""
cur.execute(cmd)
con.commit()

on supprime la table de restriction des données sur la TAN
on supprime la table de travail
on crée la table de travail en éclatant les segments
on supprime les segments de chaque ligne qui est inclus dans un buffer de 20m d'un autre segment plus long


In [2]:
print("On crée la table qui contient les segments (avec le nombre de lignes)")
cmd = "drop table tan_group;"
cur.execute(cmd)
cmd = "drop table tan_detail;"
cur.execute(cmd)

cmd="""
select geom, count(*), array_agg(code) as codes
into tan_group
from tan_tmp
group by geom
"""
cur.execute(cmd)
cmd="""
select id, geom, code, name, colour, mode, network
into tan_detail
from tan_tmp
"""
cur.execute(cmd)
con.commit()

On crée la table qui contient les segments (avec le nombre de lignes)


In [3]:
print('la requete pour trex : ')
cmd="""
select d.*, g.count, array_position(codes, d.code) as offset_order
from tan_group g left join tan_detail d
on g.geom = d.geom
"""
cur.execute(cmd)
con.commit()

la requete pour trex : 


In [4]:
con.close()
print("fin de traitement")

fin de traitement
